## Importing the associated modules

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib as plot
import sklearn as sk
from sklearn import metrics, linear_model, ensemble, neural_network, svm, dummy
from helpers import *

# Importing the different features
from ema_features import get_EMA_features_and_target_for_patient
from module_features import get_module_features_for_patient
from context_features import get_weekend_days

# Importing the machine learning module
from predicting import train_algorithms, test_algorithms, eval_algorithms, plot_algorithms, make_algorithms
from feature_selection import backward_selection, forward_selection, correlate_features, precalculated_feature_selection

# Importing multiple patient wrapper
from individual_wrappers import *

## Initializing constants

In [2]:
SLIDING_WINDOW = 7
CV_ALPHAS = (0.1, 0.3, 0.5, 0.7, 0.9)
MAX_PATIENTS = 10
FEATURE_PATH = "data/features/"

## Defining our Machine Learning Algorithms

In [3]:
ml_algorithms = [
    {
        "name": "Lasso",
        "model": linear_model.LassoCV(alphas=CV_ALPHAS)
    },
    {
        "name": "Ridge",
        "model": linear_model.RidgeCV(alphas=CV_ALPHAS)
    },
    {
        "name": "Random Forest",
        "model": ensemble.RandomForestRegressor(n_estimators=1000, max_depth=2)
    },
    {
        "name": "Dummy Mean Regressor",
        "model": dummy.DummyRegressor()
    },
    {
        "name": "SVR RBF",
        "model": svm.SVR()
    }
]

## Initializing patients

In [4]:
proper_patients = get_proper_patients(MAX_PATIENTS)

## Init-Step: Getting Features of the patients

In [5]:
sample_patient = proper_patients[:1]

sample_scores = get_patients_scores_and_features(sample_patient, ml_algorithms, 1)

/Users/jonathanmitnik/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jonathanmitnik/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jonathanmitnik/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [13]:
end_results = learn_patients_setups(sample_scores, ml_algorithms, max_features=20)

Index(['avg_count_ema_q_1_7_days', 'min_count_ema_q_1_7_days',
       'max_count_ema_q_1_7_days', 'std_count_ema_q_1_7_days',
       'avg_count_ema_q_2_7_days', 'min_count_ema_q_2_7_days',
       'max_count_ema_q_2_7_days', 'std_count_ema_q_2_7_days',
       'avg_count_ema_q_3_7_days', 'min_count_ema_q_3_7_days',
       'max_count_ema_q_3_7_days', 'std_count_ema_q_3_7_days',
       'avg_count_ema_q_4_7_days', 'min_count_ema_q_4_7_days',
       'max_count_ema_q_4_7_days', 'std_count_ema_q_4_7_days',
       'avg_count_ema_q_5_7_days', 'min_count_ema_q_5_7_days',
       'max_count_ema_q_5_7_days', 'std_count_ema_q_5_7_days',
       'avg_count_ema_q_6_7_days', 'min_count_ema_q_6_7_days',
       'max_count_ema_q_6_7_days', 'std_count_ema_q_6_7_days',
       'avg_count_ema_q_7_7_days', 'min_count_ema_q_7_7_days',
       'max_count_ema_q_7_7_days', 'std_count_ema_q_7_7_days',
       'avg_average_ema_q_1_7_days', 'min_average_ema_q_1_7_days',
       'max_average_ema_q_1_7_days', 'std_average_e

/Users/jonathanmitnik/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jonathanmitnik/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Index(['avg_count_ema_q_6_7_days', 'avg_count_ema_q_4_7_days',
       'avg_prior_engagement_7_days', 'avg_count_ema_q_7_7_days',
       'avg_count_ema_q_3_7_days', 'avg_count_ema_q_5_7_days',
       'max_count_ema_q_4_7_days', 'avg_count_ema_q_2_7_days',
       'avg_average_ema_q_4_7_days', 'max_count_ema_q_7_7_days',
       'max_count_ema_q_3_7_days', 'max_count_ema_q_6_7_days',
       'avg_count_ema_q_1_7_days', 'max_count_ema_q_5_7_days',
       'std_count_ema_q_4_7_days', 'max_prior_engagement_7_days',
       'std_count_ema_q_7_7_days', 'avg_average_ema_q_3_7_days',
       'std_count_ema_q_6_7_days', 'avg_average_ema_q_6_7_days'],
      dtype='object')


In [18]:
get_patients_mean_MAE_score(end_results[0])

2.7678084217885455